# The Joy of Integer Programming
### Solving hard combinatorial problems in python

<hr>
Charles C. Ochoa
cochoa0x1@gmail.com

slides: bitly.com/???

<hr>

<img src="img/pyconlogo-03.png" align="left"/>

# Hard problems:

* Routing
* Scheduling
* Policy
* Resource Allocation
* Network flows with funky conditions


* Optimizing a linear cost function under linear constraints

# Math Programming?


$$minimize\: f(x)\\subject\, to\: g(x) < 0 $$



# Linear Programs?

A linear program is one where the metric or objective to minimize is linear and the constraints are linear, i.e.:

$$f(x) = a_{0}x_{0} + a_{1}x_{1}+ \dots + a_{n}x_{n}$$


# Integer Programs?

Programs where the variables are integers ie, 0,1,2,3,... and some negatives.

What if I have both? Mixed Integer

# who does this?

* Very mature topic in operations research
* Computer science people
* Math people


# Lots of tools and languages exist

|Name|License|Info|
|---|---|---|
|ADMB|	BSD|	nonlinear optimization framework, using automatic differentiation|
|ALGLIB|	GPL|	nonlinear analysis library, optionally using automatic differentiation. Cross-language: C++, C#, Pascal, VBA.|
|ASCEND|	GPL|	mathematical modelling system|
|BOBYQA|	LGPL|	An algorithm that seeks the least value of a nonlinear function subject to bound constraints, without using derivatives of the objective function.|
|COBYLA|	LGPL|	An algorithm that seeks the least value of a nonlinear function subject to nonlinear inequality constraints, without using derivatives of the objective function or the constraints.|
|COIN-OR SYMPHONY|	Eclipse v.1|	integer programming|
|CUTEr|	GPL|	testing environment for optimization and linear algebra solvers|
|dlib|	Boost|	A stand-alone C++ library with a variety of linear and non-linear solvers for small and large scale problems|
|GLPK|	GPL|	GNU Linear Programming Kit|

^ small sample (source: [wikipedia](https://en.wikipedia.org/wiki/List_of_optimization_software))


# In Python?

### Use python to model/setup the problem
### Use one of the many free or commercial solvers to solve the problem

## Python PuLP package for linear mixed integer program modeling

```bash 
pip install pulp
```

# basic example 
$$minimize\: f(x,y,z)=5x+10y+6z \\
subject\, to\\
x+y+z \geq 20 \\
0\leq x,y,z \leq 10 
$$


### 1. Setup the problem

In [3]:
from pulp import *

prob = LpProblem("Hello!",LpMinimize)

### 2. Setup the variables: 

In [4]:
x = LpVariable('x',lowBound=0, upBound=10, cat='Continuous')
y = LpVariable('y',lowBound=0, upBound=10, cat='Continuous')
z = LpVariable('z',lowBound=0, upBound=10, cat='Continuous')

### 3. Setup the objective and constraints

In [5]:
#the objective
prob+= 5*x+10*y+6*z

In [6]:
#the constraint
prob+= x + y + z >= 20

### summary

In [7]:
print(prob)

Hello!:
MINIMIZE
5*x + 10*y + 6*z + 0
SUBJECT TO
_C1: x + y + z >= 20

VARIABLES
x <= 10 Continuous
y <= 10 Continuous
z <= 10 Continuous



### Solve it!

In [8]:
prob.solve()
print('status:',LpStatus[prob.status])
print('objective:', value(prob.objective))

status: Optimal
objective: 110.0


In [9]:
for v in prob.variables():
    print(v, v.varValue)

x 10.0
y 0.0
z 10.0


# Knapsack Problem

How many items of different weights can we fit in our suitcase before our suitcase is too heavy. Which objects should we take?

### 1. First lets make some fake data

In [10]:
import numpy as np

items=['item_%d'%i for i in range(20)]
item_weights = dict( (i,np.random.randint(1,20)) for i in items)
item_values = dict( (i,10*np.random.rand()) for i in items)

#target weight
W = 100

In [11]:
#create the problem
prob=LpProblem("knapsack",LpMaximize)

#variables. How many of each object to take. For simplicity lets make this 0 or 1 (classic 0-1 knapsack problem)
x = LpVariable.dicts('item',items,0,1, LpBinary)

#the objective
prob+= lpSum([ item_values[i]*x[i] for i in items])

#constraint
prob += lpSum([ item_weights[i]*x[i] for i in items]) <= W


In [12]:
prob.solve()
print(LpStatus[prob.status])

Optimal


### and the results

In [13]:
for i in items:
    if value(x[i]) > 0 : print(i, value(x[i]))

item_2 1.0
item_3 1.0
item_4 1.0
item_5 1.0
item_6 1.0
item_9 1.0
item_10 1.0
item_11 1.0
item_12 1.0
item_13 1.0
item_14 1.0
item_17 1.0
item_18 1.0
item_19 1.0


# Binary variables are awesome

### we can model basic logical operations (x,y binary)
 
|logical operation|new binary variable|
|---|---|
|not x	|$z = 1-x$|
|x or y|$ z \geq x \\z \geq y \\z \leq x+y$|
|x and y|$ z \leq x \\z \leq y \\z \geq x+y - 1$|
|$ x\implies y $|$y \geq x$|

### we can model optional enforcement of constraints, this is called the "Big M method"

#### need two constraints
if $b=1$ then $F(x) \geq 0$

if $b=0$ then $G(x) \geq 0$

#### also a big number that bounds the constraints
Find a bound $M$ such that $|F| \leq M$ and $|G| \leq M$. 

#### use this!
$$F(x) \geq -M(1-b) \\
G(x) \geq -M(b)$$

### example
$$ x \geq 10 \\ \text{or}\\ x \leq 20$$
$|x| \leq 100$ so set $M=1000$



#### result:
$$x \geq 10 - M(1-b) \\
x \leq 20 + M(b)$$

### we can model non linear functions! (kinda)

#### approximate a non linear function with a piecewise linear approximation


<img src="img/placehold_linear.png"/>


# the not joyful parts

* Debugging: Check all asssumptions, make sure problem is solveable before trying to solve it
* Form matters: The way you model the problem has a direct and sometimes dramatic influence on the running time
* Hard to know what is hard: Some problems finish in seconds, others may take hours
* Long running times:
    - accept a result within a certain percentage of the optimal bound
    - add more constraints (add cuts)
    - solve smaller problems
    - get a bigger computer

# Thanks :)


cochoa0x1@gmail.com

???.bitly.com

